In [3]:
!pip install pandarallel

  Preparing metadata (setup.py) ... done
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16672 sha256=e5dee52807b6083b2f224bffff7a354257b9bdc6689a61ad178cf133a7b38a2b
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from pandarallel import pandarallel
#pandarallel.initialize(progress_bar=True) # initialize pandarallel

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
#import plotly.express as px
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/processed/processed.pkl
/kaggle/input/test-train-3-var-no-ts/X_test.pkl
/kaggle/input/test-train-3-var-no-ts/y_train.pkl
/kaggle/input/test-train-3-var-no-ts/X_train.pkl
/kaggle/input/test-train-3-var-no-ts/y_test.pkl
/kaggle/input/later-data/Full_merged.pkl
/kaggle/input/later-data/full_clustered .pkl
/kaggle/input/later-data/cluster_model.joblib
/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet
/kaggle/input/df-mod2/df_mod2.pkl


In [ ]:
df_series=pd.read_parquet(path="/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet", engine='auto')

In [ ]:
def data_preparation(df):
    import pandas as pd
    #import numpy as np
    #from datetime import datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True,)
    df['step']=df['step'].astype(float)
    #print('.',end="")
    return(df)


In [ ]:
df_events=pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
events=df_events[['series_id', 'step','event']]

In [ ]:
df_events.info()

In [ ]:
df_events['timestamp']=df_events['timestamp'].parallel_apply(lambda x: pd.to_datetime(x, utc=True))
df_events.info()

In [ ]:
mod_series=df_series.iloc[:20000000,:]

In [ ]:
mod_series['timestamp']=mod_series['timestamp'].parallel_apply(lambda x: pd.to_datetime(x, utc=True))


In [ ]:
mod_series.info()

In [ ]:
mod_series.to_pickle('df_mod2.pkl')

In [ ]:
df=pd.merge(mod_series,events,on=["step","series_id"],how='left')

In [ ]:
series_df=pd.merge(df_series,events,on=["step","series_id"],how='left')
series_df['sleep']=np.nan
series_df.loc[series_df["event"]=="onset", "sleep"] = 1
series_df.loc[series_df["event"]=="wakeup", "sleep"] = 0
series_df['sleep'].fillna(method='ffill', inplace=True)
series_df['sleep'].fillna(value=0, inplace=True)
series_df['sleep'].mean()

In [ ]:
df['sleep']=np.nan
df.loc[df["event"]=="onset", "sleep"] = 1
df.loc[df["event"]=="wakeup", "sleep"] = 0
df['sleep'].fillna(method='ffill', inplace=True)
df['sleep'].fillna(value=0.0, inplace=True)
df['sleep'].mean()

In [ ]:
series_df.to_pickle('Full_merged.pkl')
# series_df=pd.read_pickle('/kaggle/input/processed/Full_merged.pkl')

In [ ]:
df.to_pickle('processed.pkl')

In [ ]:
df=pd.read_pickle('/kaggle/input/processed/processed.pkl')

In [ ]:
df

In [ ]:
train_df=df[['timestamp','step','anglez','enmo','sleep']]

In [ ]:
train_df

In [ ]:
import plotly.express as px
figure= px.imshow(train_df.corr(), text_auto=True, width=1200, height=1200)
figure.show()

In [ ]:
series_df

## Not significant correlation between the columns
- Let's add another column 'Cluster' based on anglez and enmo
- Let's keep the number of clusters hoping we can achive some correlation

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler    

X=series_df[['anglez','enmo']]
#Scalling the data
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)
#perform clustering
model=KMeans(n_clusters=4)
model.fit(X_scaled)

In [ ]:
X=series_df[['anglez','enmo']]
#Scalling the data
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)
#perform clustering
model=KMeans(n_clusters=4)
model.fit(X_scaled)

In [ ]:
series_df['cluster']=model.labels_
#series_df['cluster']=model.predict(series_df[['anglez','enmo']])

In [ ]:
series_df

# Saving the Clustered File

In [ ]:
series_df.to_pickle('full_clustered.pkl')

# Saving the Model

In [ ]:
from joblib import dump, load
#dump(model, 'cluster_model.joblib')
model1=clf = load(/kaggle/input/later-stage-data/cluster_model.joblib')

In [ ]:
#train_df['cluster']=model2.labels_
series_df['cluster']=model1.labels_


In [2]:
df=pd.read_pickle('/kaggle/input/later-data/full_clustered .pkl')

KeyboardInterrupt: 

In [3]:
df

NameError: name 'df' is not defined

# Train Test Split

In [10]:
from sklearn.model_selection import train_test_split
X=df[['step','anglez','enmo','cluster']]
y=df[['sleep']]
X_train, X_test, y_train, y_test =train_test_split(X,y,test_size=0.5)
#X_train.to_pickle('X_train.pkl')
#y_train.to_pickle('y_train.pkl')
#y_test.to_pickle('y_test.pkl')
#X_test.to_pickle('X_test.pkl')
df=[]

# Read from Test and Train Dataset

In [4]:
#X_train=pd.read_pickle('/kaggle/input/test-train-3-var-no-ts/X_train.pkl')
#y_train=pd.read_pickle('/kaggle/input/test-train-3-var-no-ts/y_train.pkl')
y_test=pd.read_pickle('/kaggle/input/test-train-3-var-no-ts/y_test.pkl')
X_test=pd.read_pickle('/kaggle/input/test-train-3-var-no-ts/X_test.pkl')

In [ ]:
X_train.info()

# Random Forest Classifier

## Run the model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf =RandomForestClassifier(n_jobs=2,verbose=1) 
print ('Training the model')
rf.fit(X_test,y_test)
print ('Saving the model')
from joblib import dump, load
dump(rf, 'rf_model.joblib')

In [ ]:
from sklearn.svm import SVC
svm = SVC(verbose=1) # n_neighbors is the number of neighbors to consider when predicting a class label for
svm.fit(X_test,y_test)
print ('Saving the model')
from joblib import dump, load
dump(svm, 'svm_model.joblib')

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LibSVM]....

In [10]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB() 
nb.fit(X_train,y_train)
print ('Saving the model')
from joblib import dump, load
dump(rf, 'nb_model.joblib')

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 

## Load or Save the model

## Evaluate the model

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
y_pred=rf.predict(X_test)
cm=confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True)
print("Confusion Matrix: ", cm)
print("Precision Score : ", precision_score(y_test,y_pred)) #precision measures the proportion of true positive predictions among all positive instances. how many of survived predicted actually survived, doesn't verifies 0's 70 survived as preicted whereas actually 92 survived so 70/92 will be the precision.  if we predicted 70 survived, so presion will tell how many of those 70 predicted survived matches the actual row by row data. It checkes all positives and verifies if the answer is true for each row?
print("Recall Score: ", recall_score(y_test,y_pred, average='macro')) #Recall measures the proportion of true positive predictions among all actual positive instalnces. If we predicted 100 survived correctly whereas actually 100 survived out of which 67 predicted correctly so recall will be 0.67
print("F1 Score: ",f1_score(y_test,y_pred)) 

In [ ]:
figure= px.imshow(series_df[['anglez','enmo','cluster','sleep']].corr(), text_auto=True, width=1200, height=1200)
figure.show()